# Building your first AzureML web service using Python 

This tutorial demonstrates how to deploy a machine learning model built using Python and scikit-learn. 

The tutorial uses the digits dataset that is part of the [scikit-learn distribution](http://scikit-learn.org/stable/tutorial/basic/tutorial.html).

In the tutorial, you will load a dataset, train a model on the dataset, and then publish a
realtime scoring API for the model.

To complete this tutorial:

* Sign in to your DSVM.
* Copy this notebook to a folder on the DSVM.
* In a browser, open https://&lt;your dsvm ip address&gt;:&lt;port&gt; and sign into the Jupyter server.
    * On a Linux DSVM the port is typically 8000
    * On a Windows DSVM the port is typically 9999
* In Jupyter, navigate to the folder and open the notebook.


In [47]:
# Import Azure ML API SDK. The SDK is installed implicitly with the latest
# version of the CLI in your default python environment
from azure.ml.api.schema.dataTypes import DataTypes
from azure.ml.api.schema.sampleDefinition import SampleDefinition
from azure.ml.api.realtime.services import generate_schema

In [48]:
# Read in the digits dataset
from sklearn import datasets
digits = datasets.load_digits()
print(digits.data)

[[  0.   0.   5. ...,   0.   0.   0.]
 [  0.   0.   0. ...,  10.   0.   0.]
 [  0.   0.   0. ...,  16.   9.   0.]
 ..., 
 [  0.   0.   1. ...,   6.   0.   0.]
 [  0.   0.   2. ...,  12.   0.   0.]
 [  0.   0.  10. ...,  12.   1.   0.]]


## Train your model

The task is to predict which digit an image represents in the digits dataset. There are samples of each of the 10 possible classes (the digits zero through nine) on which you *fit* an estimator to predict the classes to which unseen samples belong.

In [49]:
# Train an SVM classifier
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(digits.data[:-1], digits.target[:-1])

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [50]:
# Predict new inputs
clf.predict(digits.data[-1:])[0]

8

### Save your model

Once you have a model that performs well, you can package it into a scoring service. To prepare for this, save your model locally. You then use the Joblib library to pickle the model.

In [51]:
# Save model
from sklearn.externals import joblib
import os
if not os.path.exists('sklearn'):
    os.makedirs('sklearn')
joblib.dump(clf, 'sklearn/model.pkl')
print("Model saved")

Model saved


## Authoring a Realtime Web Service

In this section, you create and deploy a realtime web service that scores the model you saved above. 

### Define the ```init``` and ```run``` functions create the ```score.py```

Start by defining your ```init``` and ```run``` functions in the cell below. 

The ```init``` function initializes the web service, loading in any data or models that it needs to score your inputs. In the example below, it loads the trained model.

The ```run``` function defines what is executed on a scoring call. In this simple example, the service loads the json input as an array, and returns the prediction.

In [52]:
%%writefile score.py
# The init and run functions will load and score your input using the saved model.
# The score.py file will be included in the web service deployment package.
def init():   
    # read in the model file
    from sklearn.externals import joblib
    global model
    model = joblib.load('sklearn/model.pkl')
        
def run(input_array):
    try:
        if (input_array.shape != (1, 64)):
            return 'Bad input: Expecting a json encoded list of lists of shape (1,64).'
        else:
            pred = model.predict(input_array)[0]
            return int(pred)
    except Exception as e:
        return (str(e))

Overwriting score.py


### Test the ```init``` and ```run``` functions

Before publishing the web service, you can test the init and run functions in the notebook by running the the following cell.

In [53]:
import score
score.init()
score.run(digits.data[-1:])

8

### Create Web Service Schema

Using the "generate_schema" function below creates a schema JSON file for the web service. Using a schema file when creating the web service creates a Swagger document.

In [55]:
# Define the input dataframe and create the schema
dataarray = digits.data[-1:]
inputs = {"input_array": SampleDefinition(DataTypes.NUMPY, dataarray)}
generate_schema(run_func=run, inputs=inputs, filepath='service_schema.json')

{'input': {'input_array': {'internal': 'gANjYXp1cmUubWwuYXBpLnNjaGVtYS5udW1weVV0aWwKTnVtcHlTY2hlbWEKcQApgXEBfXECKFgFAAAAc2hhcGVxA0sBS0CGcQRYCQAAAGRhdGFfdHlwZXEFY251bXB5CmR0eXBlCnEGWAIAAABmOHEHSwBLAYdxCFJxCShLA1gBAAAAPHEKTk5OSv////9K/////0sAdHELYnViLg==',
   'swagger': {'example': [[0.0,
      0.0,
      10.0,
      14.0,
      8.0,
      1.0,
      0.0,
      0.0,
      0.0,
      2.0,
      16.0,
      14.0,
      6.0,
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      15.0,
      15.0,
      8.0,
      15.0,
      0.0,
      0.0,
      0.0,
      0.0,
      5.0,
      16.0,
      16.0,
      10.0,
      0.0,
      0.0,
      0.0,
      0.0,
      12.0,
      15.0,
      15.0,
      12.0,
      0.0,
      0.0,
      0.0,
      4.0,
      16.0,
      6.0,
      4.0,
      16.0,
      6.0,
      0.0,
      0.0,
      8.0,
      16.0,
      10.0,
      8.0,
      16.0,
      8.0,
      0.0,
      0.0,
      1.0,
      8.0,
      12.0,
      14.0,
      12.0,
      1.0,
      0.

### 4. Use the CLI to deploy and manage your web service

Use the following commands to set up an environment and account to deploy the web service. For more info, see the Getting Started Guide and the CLI Command Reference. You can use -h flag at the end of the commands for command help.

#### Local deployment to your machine

After setting up the local environment, use the following commands to create the web service (Sklearn folder contains the model):

```
az ml env local
az ml service create realtime -f main.py -m ../sklearn -s service_schema.json -n mydigitservice -r scikit-py
```

#### Deployment to a cluster (for production)

**Pre-requisites:**

* Create the environment (you need to do this once per environment e.g. dev or prod)

```
az ml env setup -c -n <yourclustername> --location <e.g. eastus2>
```
* Create a Model Management account (one time setup)

```
az ml account modelmanagement create --location <e.g. eastus2> -n <your-new-acctname> -g <yourresourcegroupname> --sku-capacity 1 --sku-name S1
```

*  Set the Model Management account

```
az ml account modelmanagement set -n <youracctname> -g <yourresourcegroupname>
```

*  Set the environment. The cluster name is the name used in step 1 above. The resource group name was the output of the same process and would be in the command window when the setup process is completed.

```
az ml env set -n <yourclustername> -g <yourresourcegroupname>
```

**Create and deploy the web service**

To create and run the web service on the ACS cluster:

```
az ml service create realtime -f score.py --model-file ../sklearn -s service_schema.json -n mydigitsservice -r scikit-py
```

To test the web service, run the following command with a sample data input:

```
az ml service run realtime -n mydigitservice -d "{\"input_array\": [[0.0, 0.0, 10.0, 14.0, 8.0, 1.0, 0.0, 0.0, 0.0, 2.0, 16.0, 14.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 15.0, 12.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 4.0, 16.0, 6.0, 0.0, 0.0, 8.0, 16.0, 10.0, 8.0, 16.0, 8.0, 0.0, 0.0, 1.0, 8.0, 12.0, 14.0, 12.0, 1.0, 0.0]]}"
```

To get the sample input data for the test call, use the following command:

```
az ml service show realtime -n mydigitservice
```